# Actor-Critic (AC) Method
Actor: 在这里，Actor意为“行动者”，它是整体算法本身，如果我们只用Actor而没有Critic，那么这个算法就变成了普通的policy gradient方法。它根据state输出policy，要注意，不同于输出action value，这里输出的policy是一个概率分布，所有选择的和必须等于1，这里一般用softmax函数来实现。<p>
Critic: 在这里意为“评审者”，它会尝试预测state value，一般用TD方法。它用于评估我们采取的action有多好，具体解释如下。<p>


### Actor
用于根据state选择action，即policy，一般用policy gradient方法
$$ \Delta \theta = \alpha \nabla_\theta J(\pi_\theta) = \alpha \sum_{s,a} \pi_\theta(a|s) \nabla_\theta \log \pi_\theta(a|s) b(s,a) $$
|符号|含义|
|:--|:--|
|$\theta$|policy模型参数，也是Actor的参数|
|$\alpha$|学习率|
|$J(\pi_\theta)$|policy的期望回报|
|$\nabla_\theta J(\pi_\theta)$|梯度的期望形式|
|$\pi_\theta(a\|s)$|根据state选择action的概率，即根据当前模型参数下的policy|
|$\nabla_\theta \log \pi_\theta(a\|s)$|这是policy的梯度，但是用log形式表示|
|$b(s,a)$|Baseline function|
|$\nabla_\theta \log \pi_\theta(a\|s) b(s,a)$|这是这个policy对应的state-action value的梯度，即你选了这个action对应的价值的梯度|
|$\pi_\theta(a\|s) \nabla_\theta \log \pi_\theta(a\|s) b(s,a)$|选择这个action的概率乘以这个action对应的价值的梯度，这里的policy概率起到了了一个类似权重的作用|

此公式的意义是：对于所有可能的action，对他们的价值的梯度进行加权求和，这样就可以得到policy对于这个state的总更新量，然后根据学习率进行更新。
|baseline function|含义|
|:--|:--|
|$b(s,a) = Q(s,a)$|这是最简单的baseline，就是state-action value|
|$b(s,a) = Q(s,a) - V(s)$|state-action value减去state value|
|$b(s,a) = A(s,a)$|advantage function|
|$b(s,a) = \delta$|TD error|

使用Pytorch更新过程中，我们直接这样计算梯度:
```python
...
batch_policy = policy_model(batch_state)
neg_log_prob = -torch.log(batch_policy.gather(1, batch_action.unsqueeze(1)).squeeze(1))
loss = (neg_log_prob * batch_advantage).mean()
loss.backward()
...
```
如上所示，直接用选取的action对应的policy概率乘以advantage function，然后求平均，这样就可以得到actor loss。<p>


### Critic
用于评估state-action对的价值，即value function，一般用TD方法。由于Critic输出的是state value因此采用得是如下公式：
$$ \Delta \theta = \alpha (r + \gamma V_\theta(s') - V_\theta(s))$$
|符号|含义|
|:--|:--|
|$\theta$|value function模型参数，也是Critic的参数|
|$\alpha$|学习率|
|$r$|reward|
|$\gamma$|discount|
|$V_\theta(s)$|state value|
|$r + \gamma V_\theta(s')$|是TD target，即采取了Action之后实际估算出来的state value|
|$r + \gamma V_\theta(s') - V_\theta(s)$|是TD error，就是采取了行动后估算出的return减去我们预估当前state的return|

Critic的全部意义在于计算期望差，由于我们采取行动是根据当前的policy来的，因此$r+\gamma V_\theta(s')$是我们采取行动后实际的立即收益加估算的未来收益，即当前state采取特定action后的总收益，而$V_\theta(s)$是我们旧的对当前state的总收益，两者之间的区别在于前者更实际，且受action影响。可能不巧正好采用了一个很烂的action，因此观测到后者比前者大。
